In [3]:
import ezdxf
import tkinter as tk
from tkinter import filedialog

# Definir dimensions como una variable global
cotas_vertical = []
cotas_horizontal = []

def find_dimensions(drawing):
    global cotas_vertical  # Declarar cotas_vertical como global
    global cotas_horizontal  # Declarar cotas_horizontal como global
    cotas_vertical = []
    cotas_horizontal = []

    for entity in drawing.entities:
        if entity.dxftype() == 'DIMENSION':
            actual_measurement = entity.dxf.actual_measurement
            text_midpoint_y = entity.dxf.text_midpoint[1]

            # Si el ángulo es 90 grados, se considera como cota vertical
            if entity.dxf.angle == 90:
                cotas_vertical.append((actual_measurement, text_midpoint_y))
            else:
                cotas_horizontal.append((actual_measurement, text_midpoint_y))

    # Ordenar cotas verticales de abajo hacia arriba
    cotas_vertical.sort(key=lambda x: x[1])

    # Ordenar cotas horizontales de izquierda a derecha
    cotas_horizontal.sort(key=lambda x: x[1])

    # Extraer solo los valores de las cotas ordenadas
    cotas_vertical = [cota[0] for cota in cotas_vertical]
    cotas_horizontal = [cota[0] for cota in cotas_horizontal]

    cotas_vertical.insert(0, 0)
    
def browse_file():
    filename = filedialog.askopenfilename(filetypes=[("DXF files", "*.dxf")])
    if filename:
        analyze_dxf(filename)
        print("Lista de cotas verticales:", cotas_vertical)  # Imprime la lista de cotas
        print("Lista de cotas horizontales:", cotas_horizontal)  # Imprime la lista de cotas
        root.destroy()  # Cierra la ventana principal después de procesar el archivo

def analyze_dxf(filename):
    # Abre el archivo DXF
    drawing = ezdxf.readfile(filename)

    # Encuentra las cotas y sus valores
    find_dimensions(drawing)

def main():
    global root  # Declarar root como global para poder acceder a él dentro de otras funciones
    root = tk.Tk()
    root.title("Explorador de archivos DXF")
    root.geometry("300x100")

    browse_button = tk.Button(root, text="Seleccionar archivo DXF", command=browse_file)
    browse_button.pack(pady=20)

    root.mainloop()

if __name__ == "__main__":
    main()


Lista de cotas verticales: [0, 5.0, 5.000000000000004, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.999999999999673, 4.835000882635426, 5.015000000000206, 4.999999999999658, 5.000000000000325]
Lista de cotas horizontales: [16.41999999968352]


In [4]:
import ezdxf
import math



t_chapa=["Termoacústica","Trapezoidal","Zipada"]
t_diag=["Intercalado","Igual"]

t_chapa=t_chapa[1]
t_diag=t_diag[0]


ht=7 #Altura de canaleta techo
pend=.15 #Pendiente del techo
ac=0.35 #Ancho de la canaleta embutida
sep=0.05 #Separación en union entre vigas
long_apoyo=0 #Longitud de apoyo


D=cotas_horizontal[0] #Distancia entre ejes Frontal
Le=sum(cotas_vertical) #Distancia entre ejes lateral
h=.6 #D/2/15 #Altura de montante extrema


H = D*pend/2

alpha=math.atan(2 * H / D)

sen_alpha=math.sin(alpha)
cos_alpha=math.cos(alpha)

L=D/(2*cos_alpha)
lm=h*cos_alpha


if t_chapa=="Termoacústica":
    dc=1.5
if t_chapa=="Trapezoidal":
    dc=1.2
if t_chapa=="Zipada":
    dc=1.1
    

divcorr=math.ceil(dc/(h*math.tan(math.radians(60))))

    

    
if ac!=0:
    di=(ac+lm*sen_alpha)/cos_alpha
    div=int(math.ceil((L-di-sep)/dc))
    dc=(L-di-sep)/div

else:
    div=int(math.ceil((L-sep)/dc))
    dc=(L-sep)/div
    di=d

    
divcorr=math.ceil(dc/(h*math.tan(math.radians(60)))) 
d=dc/divcorr


div=divcorr*div



if di==d:
    cant_corr=int(round((L-sep)/dc,0)) #Cantidad de correas por caída
else:
    cant_corr=int(round((L-di-sep)/dc,0)) #Cantidad de correas por caída

    
if ac!=0:
    div=div
else:
    div=div-1
    

# Crear un nuevo DXF documento en formato R2010
doc = ezdxf.new("R2010")

# Obtener el modelspace del documento
msp = doc.modelspace()

# Crear un nuevo layer
layer = doc.layers.new(name="Cordones")
layer.rgb = (255, 0, 0)  
layer = doc.layers.new(name="Montantes")
layer.rgb = (255, 255, 0)  
layer = doc.layers.new(name="Diagonales")
layer.rgb = (0, 255, 0)  
layer = doc.layers.new(name="Correas")
layer.rgb = (0, 0, 255)  
layer = doc.layers.new(name="Apoyos")
layer.rgb = (255, 0, 255)  

Cordones= []
montantes = []
diagonales = []
Correas = []
Apoyos = []
Nudos = []
    
for m in range (len(cotas_vertical)):
    # Puntos de origen
    Ox=0  
    Oy=sum(cotas_vertical[0:m+1])
    Oz=long_apoyo+ht
    if cotas_vertical[m] !=cotas_vertical[-1]:
        Sep_port = cotas_vertical[m+1]

    cordones = [[(Ox, Oy, Oz), (Ox+D/2, Oy, Oz+D*pend/2)],[(Ox, Oy, Oz+h), (Ox+D/2, Oy, Oz+D*pend/2+h)] ,  
                [(Ox+D/2, Oy, Oz+D*pend/2), (Ox+D, Oy, Oz)],[(Ox+D/2, Oy, Oz+D*pend/2+h), (Ox+D, Oy, Oz+h)] , 
                [(Ox, Oy, Oz), (Ox, Oy, Oz+h)],[(Ox+D, Oy, Oz), (Ox+D, Oy, Oz+h)], 
                [(Ox+(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha), (Ox+(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h)], 
                [(Ox+(L-sep)*cos_alpha+2*sep*cos_alpha, Oy, Oz+(L-sep)*sen_alpha), (Ox+(L-sep)*cos_alpha+2*sep*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h)]] 

    for cordon in cordones:
        Cordones.append(cordon)

    
    cm=0    
    for i in range (div):
        montantes.append([(Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(di+lm*sen_alpha)*cos_alpha-lm*sen_alpha+d*cos_alpha*cm, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+d*sen_alpha*cm)])
        montantes.append([(Ox+(D-(di+lm*sen_alpha)*cos_alpha-d*cos_alpha*cm), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(D-(di+lm*sen_alpha)*cos_alpha+lm*sen_alpha-d*cos_alpha*cm), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+d*sen_alpha*cm)])
        cm+=1

    
    if t_diag=="Intercalado":
        cm=0
        for i in range (div):
            if cm!=div-1:
                if i % 2 != 0:
                    diagonales.append([(Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(di+lm*sen_alpha)*cos_alpha-lm*sen_alpha+d*cos_alpha*(cm+1), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+d*sen_alpha*(cm+1))])
                    diagonales.append([(Ox+(D-(di+lm*sen_alpha)*cos_alpha-d*cos_alpha*cm), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(D-(di+lm*sen_alpha)*cos_alpha+lm*sen_alpha-d*cos_alpha*(cm+1)), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+d*sen_alpha*(cm+1))])
                else:
                    diagonales.append([(Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm-lm*sen_alpha, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm+lm*cos_alpha), (Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*(cm+1), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*(cm+1))])
                    diagonales.append([(Ox+(D-(di+lm*sen_alpha)*cos_alpha-d*cos_alpha*cm)+lm*sen_alpha, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm+lm*cos_alpha), (Ox+(D-(di+lm*sen_alpha)*cos_alpha-d*cos_alpha*(cm+1)), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*(cm+1))])
            else:
                if i % 2 != 0:
                    diagonales.append([(Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h)])
                    diagonales.append([(Ox+D-((di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+D-(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h)])
                else:
                    diagonales.append([(Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm-lm*sen_alpha, Oy, Oz+(di+lm*sen_alpha+d*cm)*sen_alpha+lm*cos_alpha), (Ox+di*cos_alpha+d*cos_alpha*(cm+1), Oy, Oz+di*sen_alpha+d*sen_alpha*(cm+1))])
                    diagonales.append([(Ox+D-((di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm-lm*sen_alpha), Oy, Oz+(di+lm*sen_alpha+d*cm)*sen_alpha+lm*cos_alpha), (Ox+(D-(L-sep)*cos_alpha), Oy, Oz+(L-sep)*sen_alpha)])
            
            cm+=1
            

        
    if t_diag=="Igual":
        cm=0
        for i in range (div):
            if cm!=div-1:
                    diagonales.append([(Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(di+lm*sen_alpha)*cos_alpha-lm*sen_alpha+d*cos_alpha*(cm+1), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+d*sen_alpha*(cm+1))])
                    diagonales.append([(Ox+(D-(di+lm*sen_alpha)*cos_alpha-d*cos_alpha*cm), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(D-(di+lm*sen_alpha)*cos_alpha+lm*sen_alpha-d*cos_alpha*(cm+1)), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+d*sen_alpha*(cm+1))])
            else:
                diagonales.append([(Ox+(di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h)])
                diagonales.append([(Ox+D-((di+lm*sen_alpha)*cos_alpha+d*cos_alpha*cm), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+d*sen_alpha*cm), (Ox+D-(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h)])

            cm+=1
    diagonales.append([(Ox, Oy, Oz), (Ox+(di+lm*sen_alpha)*cos_alpha-lm*sen_alpha, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha)])
    diagonales.append([(Ox+D, Oy, Oz), (Ox+(D-(di+lm*sen_alpha)*cos_alpha+lm*sen_alpha), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha)])


    
    
    if m != len(cotas_vertical)-1:

        if ac!=0:
            cm=0
            for i in range (cant_corr):
                Correas.append([(Ox+(di+lm*sen_alpha)*cos_alpha-lm*sen_alpha+dc*cos_alpha*cm, Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm), (Ox+(di+lm*sen_alpha)*cos_alpha-lm*sen_alpha+dc*cos_alpha*cm, Oy+Sep_port, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm)])
                Correas.append([(Ox+(D-(di+lm*sen_alpha)*cos_alpha+lm*sen_alpha-dc*cos_alpha*cm), Oy, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm), (Ox+(D-(di+lm*sen_alpha)*cos_alpha+lm*sen_alpha-dc*cos_alpha*cm), Oy+Sep_port, Oz+(di+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm)])
                cm+=1


        if ac==0:
            cm=0
            for i in range (cant_corr-1):
                Correas.append([(Ox+(dc+lm*sen_alpha)*cos_alpha-lm*sen_alpha+dc*cos_alpha*cm, Oy, Oz+(dc+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm), (Ox+(dc+lm*sen_alpha)*cos_alpha-lm*sen_alpha+dc*cos_alpha*cm, Oy+Sep_port, Oz+(dc+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm)])
                Correas.append([(Ox+(D-(dc+lm*sen_alpha)*cos_alpha+lm*sen_alpha-dc*cos_alpha*cm), Oy, Oz+(dc+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm), (Ox+(D-(dc+lm*sen_alpha)*cos_alpha+lm*sen_alpha-dc*cos_alpha*cm), Oy+Sep_port, Oz+(dc+lm*sen_alpha)*sen_alpha+lm*cos_alpha+dc*sen_alpha*cm)])
                cm+=1
                    
            Correas.append([(Ox, Oy, Oz+h), (Ox, Oy+Sep_port, Oz+h)])
            Correas.append([(Ox+D, Oy, Oz+h), (Ox+D, Oy+Sep_port, Oz+h)])

        Correas.append([(Ox+(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h),(Ox+(L-sep)*cos_alpha, Oy+Sep_port, Oz+(L-sep)*sen_alpha+h)])
        Correas.append([(Ox+D-(L-sep)*cos_alpha, Oy, Oz+(L-sep)*sen_alpha+h),(Ox+D-(L-sep)*cos_alpha, Oy+Sep_port, Oz+(L-sep)*sen_alpha+h)])
            
            
    
    Apoyos.append([(Ox, Oy, Oz-long_apoyo), (Ox, Oy, Oz)])
    Apoyos.append([(Ox+D, Oy, Oz-long_apoyo), (Ox+D, Oy, Oz)])

 
    
        
        
for i in range (len(Cordones)):
    Nudos.append(Cordones[i][0])
    Nudos.append(Cordones[i][1])
    
for i in range(len(montantes)):
    Nudos.append(montantes[i][0])
    Nudos.append(montantes[i][1])

# Proceso con diagonales
for i in range(len(diagonales)):
    Nudos.append(diagonales[i][0])
    Nudos.append(diagonales[i][1])

# Proceso con Correas
for i in range(len(Correas)):
    Nudos.append(Correas[i][0])
    Nudos.append(Correas[i][1])

Nudos = list(set(Nudos))
            
            
for cordon in Cordones:
    # Añadir una línea al modelspace
    line = msp.add_line(cordon[0],cordon[1])
    # Asignar la línea al nuevo layer
    line.dxf.layer = "Cordones"

for montante in montantes:    
    # Añadir una línea al modelspace
    line = msp.add_line(montante[0],montante[1])
    # Asignar la línea al nuevo layer
    line.dxf.layer = "Montantes"
    
for diagonal in diagonales:    
    # Añadir una línea al modelspace
    line = msp.add_line(diagonal[0],diagonal[1])
    # Asignar la línea al nuevo layer
    line.dxf.layer = "Diagonales"
    
for correa in Correas:    
    # Añadir una línea al modelspace
    line = msp.add_line(correa[0],correa[1])
    # Asignar la línea al nuevo layer
    line.dxf.layer = "Correas"

for apoyo in Apoyos:    
    # Añadir una línea al modelspace
    line = msp.add_line(apoyo[0],apoyo[1])
    # Asignar la línea al nuevo layer
    line.dxf.layer = "Apoyos"
    
# Guardar el documento DXF
doc.saveas("Galpon.dxf")


